# Первая задача.

2 способа распознавания:
- *нейросеть whisper от OpenAI (требует установленного ffmpeg)*
- *speech_recognition от google. ffmpeg не нужен.*

Звуковой файл подготавливаем 2 способами:
- *удаляем шум, нормализуем громкость*
- *удаляем шум, выделяем голос, нормализуем громкость (метод тяжелый)*

На выходе получаем 6 вариантов распознанного текста.

In [ ]:
# Установка необходимых библиотек
%pip install moviepy librosa noisereduce soundfile SpeechRecognition openai-whisper demucs

# Установка ffmpeg (для Windows/Linux/Mac) для whisper.
try:
    import subprocess
    subprocess.run(["ffmpeg", "-version"], check=True, capture_output=True)
    print("ffmpeg уже установлен")
except:
    print("Установка ffmpeg...")
    import platform
    if platform.system() == "Windows":
        print("Для Windows установите ffmpeg вручную: https://ffmpeg.org/download.html")
    elif platform.system() == "Linux":
        !apt-get update && apt-get install -y ffmpeg
    elif platform.system() == "Darwin":  # MacOS
        !brew install ffmpeg

In [2]:
# Указываем путь к аудиофайлу
audio_path = "task1/data/download_14.mp4"

In [ ]:
from moviepy import AudioFileClip
import os
import librosa
import noisereduce as nr
import shutil
import soundfile as sf
import speech_recognition as sr
import subprocess
import whisper


def convertation(audio_path):
    """Конвертирование звукового файла в wav."""

    # Если уже wav, то не конвертируем.
    if audio_path.lower().endswith('.wav'):
        return audio_path

    # Сохраняем файл по тому же адресу, но другим расширением.
    wav_path = os.path.splitext(audio_path)[0] + '.wav'

    audio = AudioFileClip(audio_path)
    audio.write_audiofile(wav_path, codec='pcm_s16le')
    audio.close()

    print('Файл успешно конвертирован.')
    return wav_path


def enhance_audio(audio_path):
    """Базовая функция улучшения аудио. Убирает шумы, нормализует громкость."""

    # Загрузка аудио
    audio_data, sample_rate = librosa.load(audio_path, sr=None)
    # Удаление шума
    reduced_noise = nr.reduce_noise(y=audio_data, sr=sample_rate)
    # Нормализация громкости
    normalized_audio = librosa.util.normalize(reduced_noise)
    # Сохранение обработанного аудио
    enhance_path = os.path.splitext(audio_path)[0] + '_enhance.wav'
    sf.write(enhance_path, normalized_audio, sample_rate)

    return enhance_path


def advance_enhance_audio(wav_path):
    """
    Продвинутая функция улучшения аудио.
    Убирает шумы, оставляет один голос, нормализует громкость.

    """
    # Выходной файл
    advance_enhanced_path = (
        os.path.splitext(wav_path)[0] + '_advance_enhance.wav')
    # Загрузка аудио
    audio, sample_rate = librosa.load(wav_path, sr=16000)  # 16 кГц для Whisper
    # Легкое подавление шума
    reduced_noise = nr.reduce_noise(y=audio, sr=sample_rate, stationary=False,
                                    prop_decrease=0.75)
    # Сохранение временного файла
    temp_file = "temp.wav"
    sf.write(temp_file, reduced_noise, sample_rate)
    # Выделение голоса с Demucs.
    subprocess.run(["demucs", "--two-stems=vocals", temp_file], check=True)
    vocal_file = "separated/htdemucs/temp/vocals.wav"
    # Нормализация громкости с ffmpeg
    subprocess.run([
        "ffmpeg",
        "-i", vocal_file,
        "-af", "volume=5dB",
        "-ar", "16000",
        "-y", advance_enhanced_path
    ], check=True)

    # Удаление временных файлов
    os.remove(temp_file)
    if os.path.exists("separated"):
        shutil.rmtree("separated")

    return advance_enhanced_path


def transcribe_audio(audio_path):
    """Локальное распознавание. Не требует ffmpeg."""
    recognizer = sr.Recognizer()

    print('Подождите, идет распознавание файла.')

    try:
        with sr.AudioFile(audio_path) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language="ru-RU")
            return text
    except sr.UnknownValueError:
        return "Не удалось распознать речь"
    except sr.RequestError as e:
        return f"Ошибка сервиса: {e}"


def whisper_transcribe_audio(wav_path):
    """
    Используем нейросетевую модель от OpenAI. Доступны модели tiny, base,
    small, medium, large. Использует ffmpeg (должен быть установлен в ОС).
    """
    model = whisper.load_model("base")
    result = model.transcribe(wav_path, language="ru")
    return result["text"]


# Обработка исходного файла.
def main():
    # Задаем путь к обрабатываемому аудио файлу.
    audio_path = "task1/data/download_16.mp4"

    # Конвертируем в wav.
    wav_path = convertation(audio_path)

    # Улучшаем wav 2 способами.
    enhanced_wav_path = enhance_audio(wav_path)
    advance_enhanced_wav_path = advance_enhance_audio(wav_path)

    # Распознаем текст
    texts = {
        "original": transcribe_audio(wav_path),
        "enhanced": transcribe_audio(enhanced_wav_path),
        "advance_enhanced": transcribe_audio(advance_enhanced_wav_path),
        "whisper": whisper_transcribe_audio(wav_path),
        "whisper_enhanced": whisper_transcribe_audio(enhanced_wav_path),
        "whisper_advance_enhanced": whisper_transcribe_audio(
            advance_enhanced_wav_path),
    }

    # Записываем все тексты в файлы по тому же адресу, что и оригинал.
    base_path = os.path.splitext(audio_path)[0]
    for key, text in texts.items():
        with open(f"{base_path}_{key}.txt", "w", encoding="utf-8") as file:
            file.write(text)


if __name__ == '__main__':
    main()


# Вторая задача.

Используем нейросеть yolo11n для сегментации автомобилей.
Маскируем стекла и колеса.
Вычисляем средний цвет.
Визуализируем.

In [ ]:
# Установка необходимых библиотек
%pip install opencv-python numpy matplotlib ultralytics

In [5]:
# Указываем путь к фотографии автомобиля
image_path = "task2/data/istockphoto-494522913-612x612.jpg"

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO


def detect_cars(model, image_rgb):
    """Выполняет сегментацию автомобилей с помощью модели YOLO."""
    results = model(image_rgb)
    cars = []
    masks = []
    for result in results:
        for box, mask in zip(result.boxes, result.masks or []):
            if int(box.cls) == 2:       # Класс 2 в COCO — автомобиль
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cars.append((x1, y1, x2, y2))
                mask_data = mask.data.cpu().numpy().squeeze()
                masks.append(mask_data)
    return cars, masks


def process_car(image_rgb, car_coords, mask_data):
    """
    Обрабатывает один автомобиль:
    сегментирует, исключает стёкла/колёса, вычисляет цвет.

    """
    x1, y1, x2, y2 = car_coords
    # Выделяем область автомобиля
    car_region = image_rgb[y1:y2, x1:x2].copy()

    # Изменение размера маски до размеров bounding box
    mask_resized = cv2.resize(
        mask_data, (x2 - x1, y2 - y1), interpolation=cv2.INTER_NEAREST)
    mask_resized = (mask_resized > 0).astype(np.uint8) * 255

    # Применение маски сегментации
    car_segmented = cv2.bitwise_and(car_region, car_region, mask=mask_resized)

    # Преобразование в HSV
    car_hsv = cv2.cvtColor(car_segmented, cv2.COLOR_RGB2HSV)

    # Маскирование стёкол и колёс
    lower_glass = np.array([0, 0, 150])  # Высокая яркость
    upper_glass = np.array([180, 50, 255])  # Низкая насыщенность
    lower_wheels = np.array([0, 0, 0])  # Тёмные области
    upper_wheels = np.array([180, 255, 50])  # Низкая яркость

    mask_glass = cv2.inRange(car_hsv, lower_glass, upper_glass)
    mask_wheels = cv2.inRange(car_hsv, lower_wheels, upper_wheels)
    mask_exclude = cv2.bitwise_or(mask_glass, mask_wheels)
    mask_body = cv2.bitwise_and(mask_resized, cv2.bitwise_not(mask_exclude))

    # Применение финальной маски
    car_body = cv2.bitwise_and(car_region, car_region, mask=mask_body)

    # Вычисление среднего цвета
    valid_pixels = car_body[mask_body > 0].reshape(-1, 3)
    if len(valid_pixels) > 0:
        mean_color = np.mean(valid_pixels, axis=0).astype(int)
    else:
        mean_color = np.array([128, 128, 128])

    return tuple(mean_color)


def visualize_results(image_rgb, cars, rgb_colors):
    """Визуализация."""
    # Копия изображения для рисования
    vis_image = image_rgb.copy()

    # Рисуем bounding box'ы и метки
    for i, (x1, y1, x2, y2) in enumerate(cars):
        cv2.rectangle(vis_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(vis_image, f"Car {i+1}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Визуализация
    plt.figure(figsize=(15, 5))

    # Исходное изображение
    plt.subplot(1, len(cars) + 1, 1)
    plt.imshow(vis_image)
    plt.title("Автомобили с определёнными цветами")
    plt.axis("off")

    # Цветовые патчи
    for i, color in enumerate(rgb_colors):
        plt.subplot(1, len(cars) + 1, i + 2)
        color_patch = np.ones((100, 100, 3)) * (np.array(color) / 255)
        plt.imshow(color_patch)
        plt.title(f"Авто {i+1}\nRGB: {color}")
        plt.axis("off")

    plt.tight_layout()
    plt.show()


def main():

    # Загрузка модели (vj;)
    model = YOLO("yolo11n-seg.pt")

    # Подготовка изображения
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Детекция автомобилей
    cars, masks = detect_cars(model, image_rgb)
    if not cars:
        print("Автомобили не найдены")
        return

    # Обработка автомобилей
    rgb_colors = []
    for i, (car_coords, mask_data) in enumerate(zip(cars, masks)):
        color = process_car(image_rgb, car_coords, mask_data)
        rgb_colors.append(color)
        print(f"Авто {i+1}: RGB = {color}")

    # Визуализация результатов
    visualize_results(image_rgb, cars, rgb_colors)


if __name__ == '__main__':
    main()


# Третья задача

Для решения напрашивается нейронная сеть на базе сверточной основы с обучением на сгенерированных фейковых документах с аугментацией и k-fold валидацией. Проблем с реализацией не будет, но займет существенное время. Если необходимо убедиться, что я могу это сделать, посмотрите проект https://github.com/madzone-code/FaceGenderBot
На мой взгляд, оптимальное решение в рамках тестового задания – на основе распознавания текста (писал для своего проекта). Да, оно не работает, если изображение очень низкого качества (часть предложенных паспортов). Однако, тут возникает вопрос: «а зачем такое изображение вообще классифицировать, если оно абсолютно бесполезно для дальнейшей обработки?». 
Плюс tesseract должен быть установлен в системе (прописан в PATH).

In [ ]:

# Установка необходимых библиотек
%pip install opencv-python pytesseract

# Убедитесь, что tesseract установлен в системе и прописан в PATH
# https://github.com/UB-Mannheim/tesseract/wiki

In [8]:
# Указываем путь к фотографии документа
image_path = 'task3/data/$2y$10$k1fd1.d6HmhGzjzlTay.ChRiDY8LgriFg.EupH6kUCTCt9fjRm.png'

In [ ]:
import cv2
import pytesseract


# Указываем перечень контрольных (уникальных) слов для определения типа.
DOCUMENTS_TYPE = {
    'договор': ['договор',],
    'паспорт': ['отделом', 'мвд'],
    'СТС': ['certificat'],
    'ИНН': ['налогам'],
    'права': ['водительское'],
}


# Само распознавание.
def ocr(file_path):
    """Принимаем путь к картинке и возвращаем сырые данные."""
    image = cv2.imread(file_path)
    # Конвертация изображения в градации серого.
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Конфиг для лучшего распознавания.
    custom_config = r'--oem 3 --psm 6'
    # Распознаем.
    text = pytesseract.image_to_string(
        gray_image,
        lang='rus+eng',
        config=custom_config
    )
    return set(text.lower().split())         # множество для скорости работы.


def define_type(raw_text):
    for key, values in DOCUMENTS_TYPE.items():
        for value in values:
            if value in raw_text:
                return key
    return 'Не удалось распознать документ.'


def main():
    raw_text = ocr(image_path)
    result = define_type(raw_text)
    print(result)


if __name__ == '__main__':
    main()

# Шестая задача

За основу берем решение 2 задачи. Добавляем веб-интерфейс, добавляем кеширование. Развертываем в контейнере Докер, выгружаем на докерхаб.
  
**Для развертывания контейнера выполните:**  
*docker pull madzonedocker/car-color-detection:latest*  
*docker run -p 7860:7860 --name car-color-container madzonedocker/car-color-detection:latest*  
**Для проверки перейдите по ссылке:**  
*http://localhost:7860*  


In [ ]:
# Установка необходимых библиотек
%pip install gradio

In [ ]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache
from PIL import Image
import gradio as gr
from ultralytics import YOLO

# Загрузка модели YOLO
model = YOLO('yolo11n-seg.pt')


def detect_cars(model, image_rgb):
    """Выполняет детекцию автомобилей с помощью модели YOLO."""
    results = model(image_rgb)
    cars = []
    masks = []
    for result in results:
        for box, mask in zip(result.boxes, result.masks or []):
            if int(box.cls) == 2:  # Класс 2 в COCO — автомобиль
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cars.append((x1, y1, x2, y2))
                mask_data = mask.data.cpu().numpy().squeeze()
                masks.append(mask_data)
    return cars, masks


def process_car(image_rgb, car_coords, mask_data):
    """Обрабатывает один автомобиль: сегментирует, исключает стекла/колеса,
    вычисляет цвет.
    """
    x1, y1, x2, y2 = car_coords
    car_region = image_rgb[y1:y2, x1:x2].copy()

    # Изменение размера маски до размеров bounding box
    mask_resized = cv2.resize(mask_data, (x2 - x1, y2 - y1),
                              interpolation=cv2.INTER_NEAREST)
    mask_resized = (mask_resized > 0).astype(np.uint8) * 255

    # Применение маски сегментации
    car_segmented = cv2.bitwise_and(car_region, car_region, mask=mask_resized)

    # Преобразование в HSV
    car_hsv = cv2.cvtColor(car_segmented, cv2.COLOR_RGB2HSV)

    # Маскирование стекол и колес
    lower_glass = np.array([0, 0, 150])                 # Высокая яркость
    upper_glass = np.array([180, 50, 255])              # Низкая насыщенность
    lower_wheels = np.array([0, 0, 0])                  # Темные области
    upper_wheels = np.array([180, 255, 50])             # Низкая яркость

    mask_glass = cv2.inRange(car_hsv, lower_glass, upper_glass)
    mask_wheels = cv2.inRange(car_hsv, lower_wheels, upper_wheels)
    mask_exclude = cv2.bitwise_or(mask_glass, mask_wheels)
    mask_body = cv2.bitwise_and(mask_resized, cv2.bitwise_not(mask_exclude))

    # Применение финальной маски
    car_body = cv2.bitwise_and(car_region, car_region, mask=mask_body)

    # Вычисление среднего цвета
    valid_pixels = car_body[mask_body > 0].reshape(-1, 3)
    if len(valid_pixels) > 0:
        mean_color = np.mean(valid_pixels, axis=0).astype(int)
    else:
        mean_color = np.array([128, 128, 128])

    return tuple(mean_color)


def visualize_results(image_rgb, cars, rgb_colors):
    """Визуализирует результаты: возвращает изображение."""
    vis_image = image_rgb.copy()

    # Рисуем bounding box'ы и метки
    for i, (x1, y1, x2, y2) in enumerate(cars):
        cv2.rectangle(vis_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(vis_image, f'Car {i+1}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Создаем фигуру
    plt.figure(figsize=(15, 5))

    # Исходное изображение
    plt.subplot(1, len(cars) + 1, 1)
    plt.imshow(vis_image)
    plt.title('Автомобили с определенными цветами')
    plt.axis('off')

    # Цветовые патчи
    for i, color in enumerate(rgb_colors):
        plt.subplot(1, len(cars) + 1, i + 2)
        color_patch = np.ones((100, 100, 3)) * (np.array(color) / 255)
        plt.imshow(color_patch)
        plt.title(f'Авто {i+1}\nRGB: {color}')
        plt.axis('off')

    plt.tight_layout()

    # Сохранение в буфер
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)
    image = Image.open(buf)
    return np.array(image)


@lru_cache(maxsize=100)
def cached_process_image(image_bytes, shape):
    """Кэширует обработку изображения."""
    image_rgb = np.frombuffer(image_bytes, dtype=np.uint8).reshape(shape)
    cars, masks = detect_cars(model, image_rgb)
    if not cars:
        return tuple(), tuple(), image_rgb

    rgb_colors = []
    for car_coords, mask_data in zip(cars, masks):
        color = process_car(image_rgb, car_coords, mask_data)
        rgb_colors.append(color)

    vis_image = visualize_results(image_rgb, cars, rgb_colors)
    return tuple(rgb_colors), tuple(cars), vis_image


def demo_process_image(image):
    """Обрабатывает изображение для Gradio."""
    if image is None:
        return 'Пожалуйста, загрузите изображение', None

    try:
        # Конвертация изображения в RGB
        image_rgb = np.array(image)

        # Подготовка данных для кэширования
        image_bytes = image_rgb.tobytes()
        shape = image_rgb.shape

        # Обработка с кэшированием
        rgb_colors, cars, vis_image = cached_process_image(image_bytes, shape)

        if not cars:
            return 'Автомобили не найдены', None

        # Формирование текстового результата
        result_text = '\n'.join(
            f'Авто {i+1}: RGB = {color}' for i, color in enumerate(rgb_colors)
        )

        return result_text, vis_image
    except Exception as e:
        return f'Ошибка обработки: {str(e)}', None


# Создание интерфейса Gradio
with gr.Blocks() as interface:
    gr.Markdown('# Определение цвета автомобилей')
    gr.Markdown('Загрузите изображение, чтобы определить цвета автомобилей.')
    with gr.Row():
        input_image = gr.Image(type='pil',
                               label='Загрузите фотографию автомобиля')
    with gr.Row():
        output_text = gr.Textbox(label='Цвета автомобилей (RGB)')
        output_image = gr.Image(label='Результат обработки')
    input_image.change(fn=demo_process_image,
                       inputs=input_image,
                       outputs=[output_text, output_image])

if __name__ == '__main__':
    interface.launch(server_name="0.0.0.0", server_port=7860)
